In [ ]:
%pip install selenium
%pip install webdriver-manager
%pip install pandas
%pip install lxml
%pip install openpyxl
%pip install xlrd
%pip install yfinance
%pip install XlsxWriter

## Célula 1: Importações e Configurações Iniciais


In [36]:
# Célula 1: Importações e Configurações Iniciais
import os
import zipfile
import pandas as pd
import logging
import yfinance as yf
import math
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from io import StringIO
from datetime import datetime

# Configuração de logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# Configuração das pastas
DOWNLOADS_FOLDER = "downloads"
BALANCOS_FOLDER = "balancos"
BALANCOS_XLSX_FOLDER = "balancos_xlsx"
BALANCOS_CONCATENADOS_FOLDER = "balancos_concatenados"
BALANCOS_DEFINITIVO_FOLDER = "balancos_definitivos"
COTACOES_FOLDER = "cotacoes"


# Criar pastas se não existirem
for folder in [
    DOWNLOADS_FOLDER,
    BALANCOS_FOLDER,
    BALANCOS_XLSX_FOLDER,
    BALANCOS_CONCATENADOS_FOLDER,
    BALANCOS_DEFINITIVO_FOLDER,
    COTACOES_FOLDER,
]:
    os.makedirs(folder, exist_ok=True)

## Célula 2: Configuração do WebDriver e Extração da Tabela de Resultados


In [37]:
# Célula 2: Configuração do WebDriver e Extração da Tabela de Resultados
# Configuração do WebDriver em modo headless
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)

# URL da página de resultados
url_resultado = "https://www.fundamentus.com.br/resultado.php"


# Função para obter a tabela HTML
def obter_tabela_html(url, xpath):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    elemento = driver.find_element("xpath", xpath)
    return elemento.get_attribute("outerHTML")


# Obter a tabela HTML
html_tabela = obter_tabela_html(url_resultado, "/html/body/div[1]/div[2]/table")

# Converter a tabela HTML em DataFrame
df_dados_financeiros = pd.read_html(StringIO(html_tabela), decimal=",", thousands=".")[
    0
]
df_dados_financeiros = df_dados_financeiros.sort_values(
    by="Papel", ascending=True
).reset_index(drop=True)

# Exibir o DataFrame
print("Tabela com os dados financeiros:")
display(df_dados_financeiros)

# Lista de empresas
lista_de_empresas = df_dados_financeiros["Papel"].tolist()
print("Quantidade de empresas:", len(lista_de_empresas))

# Fechar o WebDriver
driver.quit()

Tabela com os dados financeiros:


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,AALR3,15.30,-8.29,1.66,1.522,"0,00%",0.672,-11.64,35.61,-1.79,...,14.94,"4,28%","-17,52%",0.78,"2,21%","-20,06%",550839.0,1.089040e+09,0.77,"5,49%"
1,ABCB3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","14,96%",0.0,6.235070e+09,0.00,"13,50%"
2,ABCB4,22.00,5.77,0.86,0.000,"6,83%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","14,96%",16032100.0,6.235070e+09,0.00,"13,50%"
3,ABEV3,12.76,13.97,2.11,2.494,"5,72%",1.402,54.37,11.64,-22.13,...,8.07,"21,43%","18,36%",1.11,"15,82%","15,11%",358645000.0,9.526280e+10,0.04,"11,56%"
4,ABYA3,4.91,-214.80,1.76,2.055,"0,00%",0.527,1.98,19.96,-2.75,...,33.67,"10,29%","-0,96%",2.09,"2,78%","-0,82%",0.0,2.920600e+08,1.31,"16,41%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,WLMM4,31.00,8.40,1.53,0.370,"4,57%",1.046,2.79,6.34,3.48,...,6.45,"5,83%","4,40%",2.56,"19,69%","18,21%",24086.3,7.385320e+08,0.25,"28,41%"
983,WMBY3,25.39,-19.30,2.87,0.836,"0,00%",0.182,1.20,8.62,-1.50,...,25.02,"9,70%","-7,05%",1.44,"2,39%","-14,86%",0.0,2.124390e+08,6.51,"-14,48%"
984,WSON33,67.00,8.07,0.98,1.067,"0,00%",0.400,13.41,2.42,-0.89,...,5.08,"44,14%","13,58%",1.26,"18,36%","12,17%",0.0,2.148530e+09,1.21,"5,23%"
985,YDUQ3,9.60,20.27,0.94,0.559,"2,85%",0.316,3.38,3.75,-0.73,...,4.68,"14,91%","2,80%",1.68,"9,27%","4,66%",53431400.0,3.142670e+09,1.66,"9,76%"


Quantidade de empresas: 987


In [10]:
df_dados_financeiros.to_excel("dados_financeiros.xlsx", index=False)

## Célula 3: Funções Auxiliares


In [7]:
# Célula 3: Funções Auxiliares
def obter_trimestre_anterior():
    mes_atual = datetime.now().month
    ano_atual = datetime.now().year
    if mes_atual <= 3:
        return f"Q4-{ano_atual-1}"
    elif mes_atual <= 6:
        return f"Q1-{ano_atual}"
    elif mes_atual <= 9:
        return f"Q2-{ano_atual}"
    else:
        return f"Q3-{ano_atual}"


trimestre_anterior = obter_trimestre_anterior()
pasta_trimestre = os.path.join(DOWNLOADS_FOLDER, trimestre_anterior)
os.makedirs(pasta_trimestre, exist_ok=True)

## Célula 4: Download dos Arquivos


### Download dos balanços

In [ ]:
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": os.path.join(os.getcwd(), pasta_trimestre),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=options)


def baixar_e_renomear_arquivo(papel):
    arquivo_destino = os.path.join(pasta_trimestre, f"{papel}.zip")
    if os.path.isfile(arquivo_destino):
        logging.info(f"Arquivo {arquivo_destino} já existe. Pulando o papel {papel}.")
        return

    url_papeis = (
        f"https://www.fundamentus.com.br/balancos.php?papel={papel}&interface=mobile"
    )
    driver.get(url_papeis)

    try:
        time.sleep(2)
        botao_download = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "a.bt-baixar"))
        )
        botao_download.click()

        # Esperar o download começar
        for _ in range(10):
            time.sleep(5)
            if any(f.startswith("bal_") for f in os.listdir(pasta_trimestre)):
                break

        novo_arquivo = next(
            (f for f in os.listdir(pasta_trimestre) if f.startswith("bal_")), None
        )

        if novo_arquivo:
            caminho_arquivo = os.path.join(pasta_trimestre, novo_arquivo)
            os.rename(caminho_arquivo, arquivo_destino)
            logging.info(f"Arquivo {arquivo_destino} baixado e renomeado com sucesso!")
        else:
            logging.error(f"Erro ao encontrar o arquivo baixado para {papel}")

    except Exception as e:
        logging.error(f"Erro ao baixar arquivo para {papel}: {e}")


# Baixar e renomear arquivos para cada empresa
for papel in lista_de_empresas:
    baixar_e_renomear_arquivo(papel)

# Fechar o WebDriver
driver.quit()

### Obter Cotações e Salvar em Excel

In [ ]:
tickers_falhados = []
contador_de_empresas = 0

# Itera sobre a lista de empresas
for papel in lista_de_empresas:
    # Adiciona ".SA" ao nome do papel para buscar na B3
    papel_com_sufixo = papel + ".SA"
    
    # Cria o caminho do arquivo Excel
    arquivo_excel = os.path.join(COTACOES_FOLDER, f"{papel}.xlsx")

    # Verifica se o arquivo já foi baixado
    if os.path.exists(arquivo_excel):
        print(f"As cotações para {papel_com_sufixo} já foram baixadas anteriormente. Pulando...")
        contador_de_empresas += 1  # Contabiliza como sucesso, já que já existe
        continue

    try:
        print(f"Obtendo cotações para {papel_com_sufixo}...")

        # Obtém os dados de mercado usando o Yahoo Finance
        cotacao = yf.download(papel_com_sufixo, period="max")

        # Verifica se os dados foram obtidos corretamente
        if not cotacao.empty:
            # Remove as horas da coluna 'Date' e formata para 'dd/mm/yyyy'
            cotacao.index = pd.to_datetime(cotacao.index).strftime('%d/%m/%Y')

            # Transpõe o DataFrame para ter as datas como colunas
            cotacao_transposta = cotacao.transpose()

            # Escreve no Excel
            with pd.ExcelWriter(arquivo_excel, engine='xlsxwriter') as writer:
                cotacao_transposta.to_excel(writer, sheet_name='Cotações')

                # Obtém o workbook e worksheet
                workbook = writer.book
                worksheet = writer.sheets['Cotações']

                # Formata a primeira linha (as datas) no formato dd/mm/yyyy
                date_format = workbook.add_format({'num_format': 'dd/mm/yyyy'})
                worksheet.set_row(0, None, date_format)

            print(f"Cotações de {papel_com_sufixo} salvas com sucesso em {arquivo_excel}")
            contador_de_empresas += 1
        else:
            print(f"Cotações não encontradas para {papel_com_sufixo}")
            tickers_falhados.append(papel_com_sufixo)

    except Exception as e:
        print(f"Erro ao baixar cotações para {papel_com_sufixo}: {e}")
        tickers_falhados.append(papel_com_sufixo)

# Exibe o resultado
print(f"\nTotal de empresas com cotações obtidas com sucesso: {contador_de_empresas}")
if tickers_falhados:
    print(f"Empresas para as quais as cotações falharam: {', '.join(tickers_falhados)}")

## Célula 5: Extração e Renomeação dos Arquivos


In [ ]:
# Célula 5: Extração e Renomeação dos Arquivos

# Configuração básica do logging
logging.basicConfig(filename="extracao_renomeacao.log", level=logging.ERROR)


# Função de extração e renomeação
def extrair_e_renomear_arquivo(papel):
    caminho_arquivo_zip = os.path.join(pasta_trimestre, f"{papel}.zip")
    pasta_balancos_trimestre = os.path.join(BALANCOS_FOLDER, trimestre_anterior)
    os.makedirs(pasta_balancos_trimestre, exist_ok=True)
    caminho_arquivo_xls = os.path.join(pasta_balancos_trimestre, "balanco.xls")
    novo_nome_arquivo = os.path.join(pasta_balancos_trimestre, f"{papel}.xls")

    # Verifica se o arquivo já foi extraído e renomeado
    if os.path.exists(novo_nome_arquivo):
        print(f"Arquivo {novo_nome_arquivo} já foi extraído e renomeado.")
        return True  # Indica sucesso sem necessidade de renomeação

    try:
        # Extraindo o arquivo ZIP
        with zipfile.ZipFile(caminho_arquivo_zip, "r") as zip_ref:
            zip_ref.extractall(pasta_balancos_trimestre)

        # Renomeando o arquivo
        if os.path.exists(caminho_arquivo_xls):
            os.rename(caminho_arquivo_xls, novo_nome_arquivo)
            print(f"Arquivo {novo_nome_arquivo} extraído e renomeado com sucesso!")
        else:
            print(f"Arquivo balanco.xls não encontrado em {pasta_balancos_trimestre}")
            return False  # Indica falha na renomeação

    except Exception as e:
        logging.error(f"Erro ao extrair e renomear arquivo para {papel}: {e}")
        return False  # Indica falha na operação

    return True  # Indica sucesso


# Lista para armazenar papéis com erro
papeis_com_erro = []

# Extração e renomeação dos arquivos para cada empresa
for papel in lista_de_empresas[
    :
]:  # Usar uma cópia da lista para evitar problemas ao remover elementos
    sucesso = extrair_e_renomear_arquivo(papel)
    if not sucesso:
        papeis_com_erro.append(papel)

# Remover os papéis com erro da lista de empresas
for papel in papeis_com_erro:
    lista_de_empresas.remove(papel)

# Exibir os papéis que deram erro
print("PAPÉIS COM ERRO:", papeis_com_erro)
print("LISTA DE EMPRESAS ATUALIZADA:", lista_de_empresas)
print("QUANTIDADE DE EMPRESAS ATUALIZADA:", len(lista_de_empresas))

## Célula 6: Conversão de XLS para XLSX


In [ ]:
# Célula 6: Conversão de XLS para XLSX
def converter_xls_para_xlsx(papel):
    pasta_balancos_trimestre = os.path.join(BALANCOS_FOLDER, trimestre_anterior)
    pasta_balancos_xlsx_trimestre = os.path.join(
        BALANCOS_XLSX_FOLDER, trimestre_anterior
    )
    os.makedirs(pasta_balancos_xlsx_trimestre, exist_ok=True)

    caminho_xls = os.path.join(pasta_balancos_trimestre, f"{papel}.xls")
    caminho_xlsx = os.path.join(pasta_balancos_xlsx_trimestre, f"{papel}.xlsx")

    if os.path.isfile(caminho_xlsx):
        print(f"Arquivo {papel}.xlsx já foi convertido.")
        return

    try:
        xls = pd.ExcelFile(caminho_xls, engine="xlrd")
        with pd.ExcelWriter(caminho_xlsx, engine="openpyxl") as writer:
            for sheet_name in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet_name)
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Arquivo {papel}.xls convertido para {papel}.xlsx.")
    except Exception as e:
        print(f"Erro ao converter {papel}.xls: {e}")


# Converter arquivos XLS para XLSX para cada empresa
for papel in lista_de_empresas:
    converter_xls_para_xlsx(papel)

## Concatenação das Tabelas


In [ ]:
# Configurar logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# Constantes
BAL_PATRIM = "Bal. Patrim."
DEM_RESULT = "Dem. Result."
EXTENSAO_EXCEL = ".xlsx"


def reordenar_primeira_linha(df: pd.DataFrame) -> pd.DataFrame:
    """
    Reordena a primeira linha de um DataFrame, colocando as datas da mais antiga para a mais recente.
    """
    # Identificar a primeira linha (após remoção da original) para as datas
    primeira_linha = df.iloc[0, 1:]

    # Tentar converter os valores para datetime, forçando o formato "dd/mm/yyyy"
    datas = pd.to_datetime(primeira_linha, format="%d/%m/%Y", errors="coerce")

    # Verificar se todas as datas são válidas
    if datas.isnull().all():
        logging.warning(
            "Nenhuma data foi reconhecida na primeira linha. Retornando DataFrame original."
        )
        return df

    # Criar um DataFrame temporário para facilitar a reordenação
    temp_df = pd.DataFrame({"original_col": df.columns[1:], "date": datas})

    # Remover entradas que não são datas válidas
    temp_df = temp_df.dropna(subset=["date"])

    # Ordenar as colunas com base nas datas
    temp_df = temp_df.sort_values(by="date")

    # Reordenar o DataFrame original de acordo com as colunas ordenadas
    new_order = [df.columns[0]] + temp_df["original_col"].tolist()
    df = df[new_order]

    # Substituir a primeira linha com as datas ordenadas
    df.iloc[0, 1:] = temp_df["date"].dt.strftime("%d/%m/%Y").values

    return df


def processar_arquivo(caminho_arquivo: str, pasta_saida: str) -> None:
    try:
        nome_arquivo = os.path.basename(caminho_arquivo)
        nome_papel = nome_arquivo.replace(EXTENSAO_EXCEL, "")

        bal_patrim = pd.read_excel(
            caminho_arquivo, sheet_name=BAL_PATRIM, header=None, engine="openpyxl"
        )
        dem_result = pd.read_excel(
            caminho_arquivo, sheet_name=DEM_RESULT, header=None, engine="openpyxl"
        )

        # Remover a primeira linha de ambas as planilhas
        bal_patrim = bal_patrim.iloc[1:].reset_index(drop=True)
        dem_result = dem_result.iloc[1:].reset_index(drop=True)

        # Reordenar a primeira linha de ambas as planilhas
        bal_patrim = reordenar_primeira_linha(bal_patrim)
        dem_result = reordenar_primeira_linha(dem_result)

        # Verificar o conteúdo da célula [0, 0] da planilha 'Bal. Patrim.'
        primeira_celula = bal_patrim.iloc[0, 0]

        if (
            pd.to_datetime(primeira_celula, format="%d/%m/%Y", errors="coerce")
            is not pd.NaT
        ):
            # Se a célula contém uma data, mover as colunas para a direita
            bal_patrim = bal_patrim.shift(1, axis=1)

        # Definir o nome do papel na célula [0, 0] da planilha 'Bal. Patrim.'
        bal_patrim.iloc[0, 0] = nome_papel

        # Remover a primeira linha de 'Dem. Result.' antes de concatenar
        dem_result = dem_result.iloc[1:].reset_index(drop=True)

        # Concatenar as duas planilhas
        combinado = pd.concat([bal_patrim, dem_result], ignore_index=True)

        # Verificação final: Se a célula [0, 0] não contiver o nome do papel, remover a primeira linha
        if combinado.iloc[0, 0] != nome_papel:
            combinado = combinado.iloc[1:].reset_index(drop=True)

        # Criar a pasta de saída correspondente
        nome_subpasta = os.path.basename(os.path.dirname(caminho_arquivo))
        subpasta_saida = os.path.join(pasta_saida, nome_subpasta)
        os.makedirs(subpasta_saida, exist_ok=True)

        # Criar o caminho para o arquivo de saída
        arquivo_saida = os.path.join(subpasta_saida, nome_arquivo)

        # Verificar se o arquivo já existe antes de salvar
        if os.path.exists(arquivo_saida):
            logging.info(f"Arquivo já existe e será ignorado: {arquivo_saida}")
        else:
            # Salvar o resultado em uma nova planilha
            combinado.to_excel(arquivo_saida, index=False)
            logging.info(f"Arquivo combinado e ordenado salvo como: {arquivo_saida}")

    except Exception as e:
        logging.error(f"Erro ao processar o arquivo {caminho_arquivo}: {str(e)}")


def processar_todos_arquivos(pasta_base: str, pasta_saida: str) -> None:
    for nome_subpasta in sorted(os.listdir(pasta_base)):
        caminho_subpasta = os.path.join(pasta_base, nome_subpasta)
        if os.path.isdir(caminho_subpasta):
            for nome_arquivo in sorted(os.listdir(caminho_subpasta)):
                if nome_arquivo.endswith(EXTENSAO_EXCEL):
                    caminho_arquivo = os.path.join(caminho_subpasta, nome_arquivo)
                    processar_arquivo(caminho_arquivo, pasta_saida)


# Execução do processamento
processar_todos_arquivos(BALANCOS_XLSX_FOLDER, BALANCOS_CONCATENADOS_FOLDER)

In [ ]:
# Função para obter as subpastas ordenadas da mais antiga para a mais recente
def obter_subpastas_ordenadas(base_folder):
    subpastas = [f.name for f in os.scandir(base_folder) if f.is_dir()]
    subpastas.sort(
        key=lambda x: datetime.strptime(x.split("-")[1], "%Y")
    )  # Ordenar por ano
    return subpastas


# Função para concatenar as planilhas
def concatenar_planilhas(base_folder, output_folder):
    planilhas_dict = {}

    # Obter subpastas ordenadas
    subpastas_ordenadas = obter_subpastas_ordenadas(base_folder)

    # Percorre todas as subpastas (ordenadas pela data da pasta mais antiga para a mais recente)
    for subpasta in subpastas_ordenadas:
        subpasta_path = os.path.join(base_folder, subpasta)
        if os.path.isdir(subpasta_path):
            for file_name in os.listdir(subpasta_path):
                if file_name.endswith(".xlsx"):
                    file_path = os.path.join(subpasta_path, file_name)

                    # Carregar a planilha, ignorando a primeira linha (índice de colunas)
                    df = pd.read_excel(file_path, skiprows=1)

                    # Se o arquivo já foi processado, concatenar as novas colunas
                    if file_name in planilhas_dict:
                        df_existente = planilhas_dict[file_name]

                        # Ignorar a primeira coluna (indicadores financeiros)
                        for col in df.columns[1:]:
                            if col not in df_existente.columns:
                                df_existente[col] = df[col]

                        # Reordenar as colunas pela data
                        colunas_datas = [
                            col
                            for col in df_existente.columns
                            if col != df_existente.columns[0]
                        ]
                        colunas_datas.sort(
                            key=lambda date: datetime.strptime(str(date), "%d/%m/%Y")
                        )
                        colunas_ordenadas = [df_existente.columns[0]] + colunas_datas
                        planilhas_dict[file_name] = df_existente[colunas_ordenadas]
                    else:
                        # Armazenar a primeira planilha encontrada
                        planilhas_dict[file_name] = df

    # Salvar os arquivos concatenados na pasta de destino
    for nome_arquivo, df_final in planilhas_dict.items():
        # Salvar o DataFrame concatenado no arquivo de destino
        output_path = os.path.join(output_folder, nome_arquivo)
        df_final.to_excel(output_path, index=False)


# Executar a função
concatenar_planilhas(BALANCOS_CONCATENADOS_FOLDER, BALANCOS_DEFINITIVO_FOLDER)

## Dicionarios


### Dicionario de balanços


In [38]:
# Dicionário para armazenar os DataFrames
dicionario_de_balancos = {}
contador_de_empresas = 0
# Percorre todos os arquivos na pasta 'balancos_definitivos'
for arquivo in os.listdir(BALANCOS_DEFINITIVO_FOLDER):
    if arquivo.endswith(".xlsx"):
        # Cria o caminho completo para o arquivo
        caminho_arquivo = os.path.join(BALANCOS_DEFINITIVO_FOLDER, arquivo)
        contador_de_empresas = contador_de_empresas + 1
        # Carrega o DataFrame a partir do arquivo Excel
        balanco = pd.read_excel(
            caminho_arquivo, sheet_name=0, engine="openpyxl", header=None
        )

        # Na célula [0, 0] está o nome da empresa
        nome = balanco.iloc[0, 0]
        print(f"Processando empresa {nome}...")
        # Pega a 1ª linha e torna um cabeçalho
        balanco.columns = balanco.iloc[0]
        balanco = balanco[1:]

        # Define a 1ª coluna (nome da empresa) como índice
        balanco = balanco.set_index(nome)

        # Adiciona o DataFrame ao dicionário com o nome da empresa como chave
        dicionario_de_balancos[nome] = balanco
print("TOTAL DE EMPRESAS: ", contador_de_empresas)

Processando empresa HYPE3...
Processando empresa CURY3...
Processando empresa GBIO33...
Processando empresa CSRN3...
Processando empresa REDE3...
Processando empresa OIBR4...
Processando empresa AMBP3...
Processando empresa BRBI11...
Processando empresa GETT4...
Processando empresa RCSL4...
Processando empresa ENER5...
Processando empresa KLBN4...
Processando empresa ESCE3...
Processando empresa AGEN33...
Processando empresa BSLI4...
Processando empresa ARTE4...
Processando empresa TKNO4...
Processando empresa GFSA3...
Processando empresa UOLL4...
Processando empresa MEND6...
Processando empresa DOHL4...
Processando empresa DURA4...
Processando empresa CCRO3...
Processando empresa ESPA3...
Processando empresa SMFT3...
Processando empresa EUCA4...
Processando empresa CYRE3...
Processando empresa NINJ3...
Processando empresa CRDE3...
Processando empresa TTEN3...
Processando empresa IGTI3...
Processando empresa NORD3...
Processando empresa CTIP3...
Processando empresa IGBR6...
Processando

In [39]:
# Conjunto de nomes no dicionário
nomes_no_dicionario = set(dicionario_de_balancos.keys())

# Conjunto de nomes na lista de empresas
nomes_na_lista = set(lista_de_empresas)

# Empresas que estão no dicionário mas não na lista de empresas
faltando_na_lista = nomes_no_dicionario - nomes_na_lista

# Atualizar lista_de_empresas para incluir as empresas que estão no dicionário
lista_de_empresas.extend(faltando_na_lista)

# Remover duplicatas da lista_de_empresas (caso haja)
lista_de_empresas = list(set(lista_de_empresas))
sorted_lista_de_empresas = sorted(lista_de_empresas)
print("LISTA ATUALIZADA DE EMPRESAS:")
print(sorted_lista_de_empresas)

# Opcional: Exibir as empresas que estavam no dicionário, mas não na lista antes da atualização
print("Empresas que foram adicionadas à lista:")
print(faltando_na_lista)

LISTA ATUALIZADA DE EMPRESAS:
['AALR3', 'ABCB3', 'ABCB4', 'ABEV3', 'ABYA3', 'ACES3', 'ACES4', 'ADHM3', 'AEDU11', 'AEDU3', 'AELP3', 'AERI3', 'AESB3', 'AESL3', 'AESL4', 'AFLT3', 'AFLU3', 'AFLU5', 'AGEI3', 'AGEN33', 'AGIN3', 'AGRO3', 'AGXY3', 'AHEB3', 'AHEB5', 'AHEB6', 'ALBA3', 'ALLD3', 'ALLL11', 'ALLL3', 'ALLL4', 'ALOS3', 'ALPA3', 'ALPA4', 'ALPK3', 'ALSC3', 'ALSO3', 'ALUP11', 'ALUP3', 'ALUP4', 'AMAR3', 'AMBP3', 'AMBV3', 'AMBV4', 'AMER3', 'AMIL3', 'AMPI3', 'ANIM3', 'APER3', 'APTI4', 'ARCE3', 'ARCZ3', 'ARCZ6', 'ARLA3', 'ARLA4', 'ARML3', 'ARPS3', 'ARPS4', 'ARTE3', 'ARTE4', 'ARTR3', 'ARZZ3', 'ASAI3', 'ASSM3', 'ASSM4', 'ASTA4', 'ATMP3', 'ATOM3', 'AURA33', 'AURE3', 'AUTM3', 'AVIL3', 'AVLL3', 'AZEV3', 'AZEV4', 'AZUL4', 'AZZA3', 'B3SA3', 'BAHI11', 'BAHI3', 'BAHI4', 'BAHI5', 'BALM3', 'BALM4', 'BAUH4', 'BAZA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBRK3', 'BBSE3', 'BBTG11', 'BBTG12', 'BBTG13', 'BCAL6', 'BDLL3', 'BDLL4', 'BECE3', 'BECE4', 'BEEF3', 'BEES3', 'BEES4', 'BELG3', 'BELG4', 'BEMA3', 'BERG3', 'BESP3

In [40]:
# Configurações
data_referencia = "01/01/2023"
data_referencia_aux = pd.to_datetime(data_referencia, format="%d/%m/%Y")

# Lista para armazenar empresas a serem removidas
lista_para_remover = []

# Filtra empresas com último balanço anterior à data de referência
for papel, balanco in dicionario_de_balancos.items():
    colunas = list(balanco.columns)

    try:
        # Converte a última coluna para datetime, especificando o formato e dayfirst=True
        ultima_data_aux = pd.to_datetime(colunas[-1], format="%d/%m/%Y", dayfirst=True)

        if ultima_data_aux <= data_referencia_aux:
            lista_para_remover.append(papel)
    except Exception as e:
        print(f"Erro ao processar a data para {papel}: {e}")

# Exibe a lista de empresas que serão removidas
print("LISTA DE EMPRESAS A SEREM REMOVIDAS: ", lista_para_remover)

# Remove as empresas do dicionário
for papel in lista_para_remover:
    dicionario_de_balancos.pop(papel)

# Exibe o total remanescente e a lista atualizada de empresas
print("\nTOTAL REMANESCENTE:", len(dicionario_de_balancos.keys()))

LISTA DE EMPRESAS A SEREM REMOVIDAS:  ['GBIO33', 'GETT4', 'AGEN33', 'ARTE4', 'UOLL4', 'DURA4', 'CRDE3', 'CTIP3', 'IGBR6', 'DJON4', 'DOCA3', 'PRVI3', 'TNLP3', 'ELEV3', 'NETC4', 'FGUI3', 'BESP3', 'LATS3', 'BMEF3', 'PEFX5', 'FCAP3', 'LECO3', 'BHGR3', 'LETO5', 'BRTP3', 'PTIP3', 'BPIA3', 'PTQS4', 'MMXM3', 'IGUA5', 'LETO3', 'IDVL4', 'BBSE3', 'NAFG4', 'BEMA3', 'BTTL3', 'SGPS3', 'TROR4', 'JBDU4', 'TSEP3', 'FBMC4', 'VPSC3', 'VIIA3', 'PNOR5', 'TNCP3', 'ARZZ3', 'REEM4', 'CZRS4', 'UBBR3', 'CRUZ3', 'FBMC3', 'IMCH3', 'TCNO4', 'BLUT4', 'BSCT6', 'LCSA4', 'CZRS3', 'TOYB3', 'RSIP4', 'ARTE3', 'BSGR3', 'ARLA3', 'CCTU4', 'RNPT3', 'WIZS3', 'AGEI3', 'BNCA3', 'CPNY3', 'MRSL3', 'TMCP3', 'UBBR4', 'JFAB4', 'TIET11', 'REPA3', 'CIQU4', 'BRDT3', 'PQUN3', 'CZLT33', 'TMAR5', 'VPTA4', 'BUET4', 'RAIA3', 'AELP3', 'SPRI5', 'TAMM4', 'LFFE4', 'IDVL11', 'DHBI3', 'TSPP3', 'SPRI6', 'CRTP3', 'DMMO3', 'AMBV4', 'AEDU3', 'TEFC11', 'VIGR3', 'VAGV4', 'SMLE3', 'SGEN4', 'TRFO3', 'SEBB4', 'FCAP4', 'BECE3', 'TEMP3', 'IVTT3', 'BFIT4', '

In [41]:
dicionario_de_balancos["PETR4"]

,30/06/2009,30/09/2009,31/12/2009,31/03/2010,30/06/2010,30/09/2010,31/12/2010,31/03/2011,30/06/2011,30/09/2011,...,31/12/2021,31/03/2022,30/06/2022,30/09/2022,31/12/2022,31/03/2023,30/06/2023,30/09/2023,31/12/2023,31/03/2024
PETR4,,,,,,,,,,,,,,,,,,,,,
Ativo Total,305265344.512,333789528.064,345607274.496,365998080,382029856.768,507697037.312,519970029.568,544945274.88,554583457.792,582124896.256,...,972950994.944,998662012.928,1004753977.344,947574013.952,976708960.256,978576998.4,990459002.88,1025495990.272,1050887979.008,1067292033.024
Ativo Circulante,57621536.768,75719057.408,76674015.232,74459103.232,71980236.8,111415033.856,106685161.472,120036376.576,119493476.352,120493400.064,...,168247001.088,199511998.464,224784990.208,159327010.816,163051995.136,157193994.24,136816001.024,147311001.6,157079011.328,165964005.376
Caixa e Equivalentes de Caixa,10072162.304,30088286.208,28795713.536,26951325.696,24209866.752,47291932.672,30323259.392,43344818.176,34672615.424,33659195.392,...,58410000.384,81601003.52,85310996.48,23650000.896,41722998.784,52276998.144,49882001.408,60642000.896,61612998.656,57689001.984
Aplicações Financeiras,0,0,0,0,0,0,26017296.384,20015771.648,24969263.104,21410791.424,...,3630000.128,5967000.064,14956999.68,13038000.128,14469999.616,14629000.192,11102999.552,6504999.936,13649999.872,24071999.488
Contas a Receber,14555268.096,13643311.104,13984270.336,16200354.816,15961581.568,18407325.696,17333975.04,17777672.192,18762930.176,20466372.608,...,35538001.92,26848999.424,26442999.808,22026000.384,26141999.104,23497000.96,21041000.448,25501999.104,29702000.64,25184000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IR Diferido,-679049.024,-1329499.008,446287.008,-1541420.032,-2775495.936,-1923373.952,-2373476.096,-2373476.096,-1750392.064,925491.968,...,-10135000.064,-136000,-1222000,6975000.064,-3497999.872,-4003000.064,1504999.936,1453999.872,1656999.936,5568000
Participações/Contribuições Estatutárias,0,-1495323.008,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Reversão dos Juros sobre Capital Próprio,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dicionario de cotações


In [ ]:
# Dicionário para armazenar as cotações
dicionario_de_cotacoes = {}
tickers_falhados = []
keys_list = list(dicionario_de_balancos.keys())

# Itera sobre as chaves do dicionário de balanços
for papel in keys_list:
    # Adiciona ".SA" ao nome do papel para buscar na B3
    papel_com_sufixo = papel + ".SA"

    try:
        print(f"Obtendo cotações para {papel_com_sufixo}...")

        # Obtém os dados de mercado usando o Yahoo Finance
        cotacao = yf.download(papel_com_sufixo)

        if not cotacao.empty:
            # Seleciona as colunas desejadas e organiza o DataFrame
            cotacao_filtrada = cotacao[["Open", "High", "Low", "Close", "Adj Close"]]
            cotacao_filtrada.columns = [
                "Abertura",
                "Máxima",
                "Baixa",
                "Fechamento",
                "Fechamento Ajustado",
            ]

            # Transpõe o DataFrame para que as colunas virem linhas e as datas sejam colunas
            cotacao_transposta = cotacao_filtrada.transpose()

            # Adiciona o DataFrame transposto ao dicionário de cotações
            dicionario_de_cotacoes[papel] = cotacao_transposta
        else:
            print(f"Cotações não encontradas para {papel_com_sufixo}")
            tickers_falhados.append(papel_com_sufixo)

    except Exception as e:
        print(f"Erro ao baixar cotações para {papel_com_sufixo}: {e}")
        tickers_falhados.append(papel_com_sufixo)

# Exibe o resultado
print("\nTOTAL DE COTAÇÕES OBTIDAS:", len(dicionario_de_cotacoes.keys()))
print("\nCOTAÇÕES OBTIDAS PARA OS SEGUINTES TICKERS:", dicionario_de_cotacoes.keys())
print("COTAÇÕES NÃO ENCONTRADAS PARA OS SEGUINTES TICKERS:", tickers_falhados)

In [54]:
# Exibe o resultado
print(f"\nTotal de empresas com cotações obtidas com sucesso: {contador_de_empresas}")
sorted_tickers_falhados = sorted(tickers_falhados)
if sorted_tickers_falhados:
    print(f"Empresas para as quais as cotações falharam: {', '.join(tickers_falhados)}")


Total de empresas com cotações obtidas com sucesso: 518
Empresas para as quais as cotações falharam: EBEN4.SA, VTLM3.SA, DFVA4.SA, FFTL3.SA, BECE3.SA, ROMI4.SA, REPA3.SA, LAME3.SA, PLTO6.SA, ARCZ6.SA, BSCT6.SA, WHIRL4.SA, RPSA4.SA, LETO5.SA, CSAB3.SA, BMGB11.SA, BIDI3.SA, SUZB5.SA, VGOR3.SA, ASSM4.SA, ABCB3.SA, PLDN4.SA, BOVH3.SA, GETI4.SA, RSIP3.SA, VIGR3.SA, UGPA4.SA, RSIP4.SA, WEGE4.SA, BSCT3.SA, CEPE6.SA, ALBA3.SA, TMGC3.SA, PEFX5.SA, TMGC7.SA, GALO4.SA, TLCP4.SA, PTQS4.SA, TCOC4.SA, BIDI4.SA, CTPC4.SA, ELPL5.SA, CALI4.SA, UOLL4.SA, ABYA3.SA, BRSR4.SA, EEEL3.SA, PALF5.SA, PTPA4.SA, SUZA4.SA, CTSA8.SA, LECO4.SA, CAMB4.SA, IVTT3.SA, TPRC6.SA, TBLE6.SA, ARTE4.SA, WSON33.SA, OSAO4.SA, TCNO3.SA, BELG4.SA, IMCH3.SA, EQMA5B.SA, PCAR5.SA, ILMD3.SA, VINE3.SA, FLCL5.SA, TIET3.SA, TUPY4.SA, BRTP3.SA, GAFP3.SA, ESTC4.SA, MODL4.SA, CRBM7.SA, TPRC3.SA, ILMD4.SA, SUBA3.SA, GUAR4.SA, CMET4.SA, UBBR3.SA, SALM4.SA, BBTG12.SA, TMCP4.SA, FGUI3.SA, IENG5.SA, BTTL3.SA, SASG3.SA, CGOS4.SA, VAGV3.SA, SQI

In [11]:
# Configurações
data_referencia = "2023/01/01"
data_referencia_aux = pd.to_datetime(data_referencia)

# Lista para armazenar empresas a serem removidas
lista_para_remover = []

# Filtra empresas com última cotação anterior à data de referência
for papel, cotacao in dicionario_de_cotacoes.items():
    try:
        # Extraindo as datas das colunas do DataFrame transposto
        datas = cotacao.columns

        # Verifica se há datas disponíveis
        if not datas.empty:
            # Converte a última data para datetime
            ultima_data_aux = pd.to_datetime(datas[-1])

            # Compara a última data com a data de referência
            if ultima_data_aux <= data_referencia_aux:
                lista_para_remover.append(papel)
        else:
            print(f"Sem dados de datas para {papel}.")
            lista_para_remover.append(papel)

    except Exception as e:
        print(f"Erro ao processar a data para {papel}: {e}")
        lista_para_remover.append(papel)

# Exibe a lista de empresas que serão removidas
print("LISTA DE EMPRESAS A SEREM REMOVIDAS: ", lista_para_remover)

# Remove as empresas do dicionário
for papel in lista_para_remover:
    dicionario_de_cotacoes.pop(papel, None)

# Exibe o total remanescente e a lista atualizada de empresas
print("\nTOTAL REMANESCENTE:", len(dicionario_de_cotacoes.keys()))
print("\nEMPRESAS REMANESCENTES:", list(dicionario_de_cotacoes.keys()))

LISTA DE EMPRESAS A SEREM REMOVIDAS:  ['VALE5', 'SJOS4', 'CBMA3', 'SULT4', 'SCLO4', 'CBMA4', 'SJOS3', 'SULT3', 'VVAR4', 'VVAR11', 'SCLO3', 'KROT11', 'ELPL4']

TOTAL REMANESCENTE: 466

EMPRESAS REMANESCENTES: ['HYPE3', 'CURY3', 'CSRN3', 'REDE3', 'OIBR4', 'AMBP3', 'BRBI11', 'RCSL4', 'KLBN4', 'BSLI4', 'TKNO4', 'GFSA3', 'DOHL4', 'CCRO3', 'ESPA3', 'SMFT3', 'EUCA4', 'CYRE3', 'NINJ3', 'TTEN3', 'IGTI3', 'NORD3', 'NGRD3', 'MDIA3', 'BRGE7', 'JFEN3', 'CEDO3', 'BMEB3', 'TGMA3', 'ENEV3', 'UNIP5', 'AHEB6', 'SAPR3', 'BAHI3', 'ESTR3', 'ENGI11', 'RPAD3', 'VSTE3', 'MERC3', 'AURA33', 'AERI3', 'AMER3', 'AURE3', 'JALL3', 'BRFS3', 'BRIV4', 'HETA4', 'ECOR3', 'PINE4', 'TASA3', 'ITSA3', 'MYPK3', 'BOBR3', 'RADL3', 'ALUP4', 'PSSA3', 'FHER3', 'ARML3', 'LUXM4', 'TEKA4', 'SLED4', 'CSRN6', 'BEES4', 'EGIE3', 'LPSB3', 'BRIT3', 'CBAV3', 'BPAC3', 'SBSP3', 'MELK3', 'ITUB3', 'WLMM4', 'PTBL3', 'PEAB3', 'WHRL3', 'CORR4', 'DASA3', 'POMO3', 'UCAS3', 'SUZB3', 'CRFB3', 'NEOE3', 'QUAL3', 'AZUL4', 'VITT3', 'RNEW11', 'APTI4', 'BRS

In [12]:
dicionario_de_cotacoes["AALR3"]

Date,2016-10-28,2016-10-31,2016-11-01,2016-11-03,2016-11-04,2016-11-07,2016-11-08,2016-11-09,2016-11-10,2016-11-11,...,2024-09-10,2024-09-11,2024-09-12,2024-09-13,2024-09-16,2024-09-17,2024-09-18,2024-09-19,2024-09-20,2024-09-23
Abertura,19.299999,19.190001,18.059999,18.000000,17.950001,17.98000,17.379999,17.000000,16.940001,15.81000,...,15.39,15.59,15.830000,15.660000,16.120001,16.200001,15.64,15.29,14.83,15.42
Máxima,19.760000,19.200001,18.379999,18.240000,18.120001,18.00000,17.790001,17.000000,16.940001,15.81000,...,15.58,15.83,16.200001,16.200001,16.180000,16.200001,15.64,15.49,15.44,15.42
Baixa,18.920000,17.510000,17.160000,17.309999,17.709999,17.41000,17.110001,15.700000,15.180000,14.53000,...,15.10,15.41,15.650000,15.660000,16.000000,15.670000,14.57,14.51,14.55,14.64
Fechamento,19.200001,18.059999,17.900000,17.990000,17.750000,17.65000,17.490000,16.549999,15.980000,15.25000,...,15.58,15.83,15.650000,16.129999,16.170000,15.740000,15.30,15.00,15.44,14.98
Fechamento Ajustado,19.007032,17.878487,17.720097,17.809193,17.571604,17.47261,17.314219,16.383665,15.819393,15.09673,...,15.58,15.83,15.650000,16.129999,16.170000,15.740000,15.30,15.00,15.44,14.98


### Chaves comuns entre os dicionários


In [13]:
# Obtém as chaves de ambos os dicionários
chaves_balancos = set(dicionario_de_balancos.keys())
chaves_cotacoes = set(dicionario_de_cotacoes.keys())

# Encontra as chaves comuns
chaves_comuns = chaves_balancos.intersection(chaves_cotacoes)

# Remove as entradas dos dicionários que não estão nas chaves comuns
for chave in list(dicionario_de_balancos.keys()):
    if chave not in chaves_comuns:
        del dicionario_de_balancos[chave]

for chave in list(dicionario_de_cotacoes.keys()):
    if chave not in chaves_comuns:
        del dicionario_de_cotacoes[chave]

print("Dicionários atualizados!")
print(f"Total de empresas restantes nos balanços: {len(dicionario_de_balancos.keys())}")
print(f"Total de empresas restantes nas cotações: {len(dicionario_de_cotacoes.keys())}")

Dicionários atualizados!
Total de empresas restantes nos balanços: 466
Total de empresas restantes nas cotações: 466


## Graham e PEG


### Graham


In [14]:
# Função para calcular o valor de Graham
def calcular_graham(lpa, vpa):
    return math.sqrt(22.5 * lpa * vpa) if lpa > 0 and vpa > 0 else None


# Função para pegar o número de ações do Yahoo Finance
def obter_numero_acoes(papel):
    ticker = yf.Ticker(f"{papel}.SA")
    numero_acoes = ticker.info.get("sharesOutstanding", None)  # Número de ações
    return numero_acoes


# Transformar as chaves do dicionário em uma lista
lista_empresas = list(dicionario_de_balancos.keys())

# Dicionário para armazenar os DataFrames de Graham
dicionario_de_graham = {}
contador_de_empresas = 0

# Lista para armazenar empresas com erro
empresas_com_erro = []

# Percorre a lista de empresas no dicionário
for empresa in lista_empresas:
    arquivo = f"{empresa}.xlsx"  # Formato esperado dos arquivos na pasta
    caminho_arquivo = os.path.join(BALANCOS_DEFINITIVO_FOLDER, arquivo)

    # Verifica se o arquivo existe
    if os.path.exists(caminho_arquivo):
        try:
            # Carrega o DataFrame a partir do arquivo Excel
            balanco = pd.read_excel(
                caminho_arquivo, sheet_name=0, engine="openpyxl", header=None
            )

            # Na célula [0, 0] está o nome da empresa (mesmo que já saibamos da lista)
            nome = balanco.iloc[0, 0]
            print(f"Processando empresa {nome}...")

            # Pega a 1ª linha e torna um cabeçalho
            balanco.columns = balanco.iloc[0]
            balanco = balanco[1:]

            # Define a 1ª coluna (nome da empresa) como índice
            balanco = balanco.set_index(balanco.columns[0])

            # Pega o número de ações da empresa
            numero_acoes = obter_numero_acoes(nome)

            if numero_acoes:
                # Inicializa dicionários para armazenar LPA, VPA e Graham
                lpa_dict = {}
                vpa_dict = {}
                graham_dict = {}

                # Itera sobre as colunas de datas
                for coluna in balanco.columns[1:]:
                    try:
                        # Pega o Patrimônio Líquido e o Lucro Líquido para a data
                        patrimonio_liquido = (
                            balanco.loc["Patrimônio Líquido", coluna] * 1000
                        )
                        lucro_liquido = (
                            balanco.loc["Lucro/Prejuízo do Período", coluna] * 1000
                        )

                        # Calcula o LPA e o VPA
                        lpa = lucro_liquido / numero_acoes
                        vpa = patrimonio_liquido / numero_acoes

                        # Calcula o valor de Graham
                        valor_graham = calcular_graham(lpa, vpa)

                        # Armazena LPA, VPA e Graham em seus respectivos dicionários
                        lpa_dict[coluna] = lpa
                        vpa_dict[coluna] = vpa
                        graham_dict[coluna] = valor_graham

                        contador_de_empresas += 1

                    except Exception as e:
                        print(
                            f"Erro ao calcular Graham para {nome} na data {coluna}: {e}"
                        )
                        empresas_com_erro.append(nome)

                # Cria o DataFrame com LPA, VPA e Graham como índice e datas como colunas
                df_graham = pd.DataFrame(
                    {"LPA": lpa_dict, "VPA": vpa_dict, "Valor de Graham": graham_dict}
                ).T  # Transpõe o DataFrame para que as datas sejam colunas

                # Adiciona o DataFrame ao dicionário principal
                dicionario_de_graham[nome] = df_graham

            else:
                print(f"Erro ao obter número de ações para {nome}")
                empresas_com_erro.append(nome)

        except Exception as e:
            print(f"Erro ao processar a empresa {empresa}: {e}")
            empresas_com_erro.append(nome)

    else:
        print(f"Arquivo {arquivo} não encontrado.")
        empresas_com_erro.append(empresa)

# Exibe o total de empresas processadas
print("TOTAL DE EMPRESAS: ", contador_de_empresas)
print(f"TOTAL DE EMPRESAS NO DICIONÁRIO DE GRAHAM: {len(dicionario_de_graham.keys())}")

# Exibe a lista de empresas que deram erro
if empresas_com_erro:
    print("Empresas que apresentaram erro:", empresas_com_erro)
else:
    print("Nenhuma empresa apresentou erro.")

Processando empresa HYPE3...
Processando empresa CURY3...
Processando empresa CSRN3...
Processando empresa REDE3...
Processando empresa OIBR4...
Processando empresa AMBP3...
Processando empresa BRBI11...
Erro ao obter número de ações para BRBI11
Processando empresa RCSL4...
Processando empresa KLBN4...
Processando empresa BSLI4...
Processando empresa TKNO4...
Processando empresa GFSA3...
Processando empresa DOHL4...
Processando empresa CCRO3...
Processando empresa ESPA3...
Processando empresa SMFT3...
Processando empresa EUCA4...
Processando empresa CYRE3...
Processando empresa NINJ3...
Processando empresa TTEN3...
Processando empresa IGTI3...
Processando empresa NORD3...
Processando empresa NGRD3...
Processando empresa MDIA3...
Processando empresa BRGE7...
Erro ao obter número de ações para BRGE7
Processando empresa JFEN3...
Processando empresa CEDO3...
Processando empresa BMEB3...
Processando empresa TGMA3...
Processando empresa ENEV3...
Processando empresa UNIP5...
Processando empre

In [15]:
dicionario_de_graham["PETR4"]

,30/09/2009,31/12/2009,31/03/2010,30/06/2010,30/09/2010,31/12/2010,31/03/2011,30/06/2011,30/09/2011,31/12/2011,...,31/12/2021,31/03/2022,30/06/2022,30/09/2022,31/12/2022,31/03/2023,30/06/2023,30/09/2023,31/12/2023,31/03/2024
LPA,1.492495,1.418576,1.522995,1.572836,1.946507,2.016885,2.016885,2.009169,1.163301,0.927068,...,8.181584,9.975214,8.463417,7.957589,7.005600,5.284495,4.888461,5.699623,4.351419,-0.478289
VPA,28.530560,29.278361,31.267619,32.493187,54.596712,56.323363,57.795203,59.251186,60.296482,60.559789,...,71.115210,79.719821,75.377767,68.392177,66.573761,73.824289,68.058936,70.872487,69.850549,74.843659
Valor de Graham,30.953005,30.569684,32.733153,33.910107,48.899283,50.556398,51.212707,51.754485,39.726775,35.541783,...,114.417174,133.762852,119.807984,110.658611,102.439036,93.689870,86.520674,95.335174,82.697352,NaN


### PEG


### PEG com Erro


In [16]:
# Função para calcular o PEG Ratio
def calcular_peg(preco, lpa, crescimento_lucro):
    try:
        # Evita divisões por zero ou valores negativos
        if (
            isinstance(lpa, (int, float))
            and isinstance(crescimento_lucro, (int, float))
            and lpa > 0
            and crescimento_lucro > 0
        ):
            return (preco / lpa) / crescimento_lucro
        else:
            return None
    except Exception as e:
        print(f"Erro ao calcular PEG: {e}")
        return None


# Função para converter valores em float, lidando com possíveis erros
def converter_para_float(valor):
    try:
        return float(valor)
    except ValueError:
        return None


# Função para pegar o preço de fechamento do dicionário de cotações
def obter_preco_fechamento(papel, data):
    try:
        # Verifica se o papel está no dicionário de cotações
        if papel in dicionario_de_cotacoes:
            cotacoes = dicionario_de_cotacoes[papel]
            data_formatada = pd.to_datetime(data, format="%d/%m/%Y", errors="coerce")

            if data in cotacoes.columns:
                return float(cotacoes.loc["Fechamento", data])
            else:
                datas_disponiveis = pd.to_datetime(cotacoes.columns, errors="coerce")
                datas_disponiveis = datas_disponiveis[
                    datas_disponiveis <= data_formatada
                ]

                if not datas_disponiveis.empty:
                    ultima_data = datas_disponiveis[-1]
                    return float(cotacoes.loc["Fechamento", str(ultima_data.date())])
                else:
                    print(f"Nenhuma data anterior encontrada para {papel}.")
                    return None
        else:
            print(f"Cotações não encontradas para {papel}.")
            return None
    except Exception as e:
        print(f"Erro ao obter preço de fechamento para {papel} na data {data}: {e}")
        return None


# Transformar as chaves do dicionário em uma lista
# lista_de_empresas = list(dicionario_de_balancos.keys())
# sorted_lista_de_empresas = sorted(lista_de_empresas)
sorted_lista_de_empresas = ["AALR3"]

# Dicionário para armazenar os DataFrames de PEG
dicionario_de_peg = {}
contador_de_empresas = 0

# Lista para armazenar empresas com erro
empresas_com_erro = []

# Percorre a lista de empresas no dicionário
for empresa in sorted_lista_de_empresas:
    arquivo = f"{empresa}.xlsx"
    caminho_arquivo = os.path.join(BALANCOS_DEFINITIVO_FOLDER, arquivo)

    if os.path.exists(caminho_arquivo):
        try:
            balanco = pd.read_excel(
                caminho_arquivo, sheet_name=0, engine="openpyxl", header=None
            )

            nome = balanco.iloc[0, 0]
            print(f"Processando empresa {nome}...")

            balanco.columns = balanco.iloc[0]
            balanco = balanco[1:]
            balanco = balanco.set_index(balanco.columns[0])

            lpa_dict = {}
            crescimento_dict = {}
            peg_dict = {}

            erro_na_empresa = False  # Flag para evitar erros repetidos

            # Itera sobre as colunas de datas
            for coluna in balanco.columns[1:]:
                try:
                    lucro_liquido = (
                        converter_para_float(
                            balanco.loc["Lucro/Prejuízo do Período", coluna]
                        )
                        * 1000
                    )
                    lucro_liquido_anterior = (
                        converter_para_float(
                            balanco.loc[
                                "Lucro/Prejuízo do Período",
                                balanco.columns[balanco.columns.get_loc(coluna) - 1],
                            ]
                        )
                        * 1000
                        if coluna > 1
                        else None
                    )

                    if lucro_liquido is None or (
                        lucro_liquido_anterior is not None
                        and lucro_liquido_anterior <= 0
                    ):
                        erro_na_empresa = True
                        break

                    numero_acoes = obter_numero_acoes(nome)
                    if not numero_acoes:
                        erro_na_empresa = True
                        break

                    lpa = lucro_liquido / numero_acoes
                    crescimento_lucro = (
                        (
                            (lucro_liquido - lucro_liquido_anterior)
                            / abs(lucro_liquido_anterior)
                        )
                        if lucro_liquido_anterior
                        else None
                    )

                    preco_acao = obter_preco_fechamento(empresa, coluna)
                    if preco_acao is None:
                        erro_na_empresa = True
                        break

                    valor_peg = calcular_peg(preco_acao, lpa, crescimento_lucro)

                    lpa_dict[coluna] = lpa
                    crescimento_dict[coluna] = crescimento_lucro
                    peg_dict[coluna] = valor_peg

                except Exception as e:
                    print(f"Erro ao calcular PEG para {nome} na data {coluna}: {e}")
                    erro_na_empresa = True
                    break

            if erro_na_empresa:
                empresas_com_erro.append(nome)
            else:
                df_peg = pd.DataFrame(
                    {
                        "LPA": lpa_dict,
                        "Crescimento do Lucro": crescimento_dict,
                        "PEG": peg_dict,
                    }
                ).T
                dicionario_de_peg[nome] = df_peg
                contador_de_empresas += 1

        except Exception as e:
            print(f"Erro ao processar a empresa {empresa}: {e}")
            empresas_com_erro.append(nome)

    else:
        print(f"Arquivo {arquivo} não encontrado.")
        empresas_com_erro.append(empresa)

# Exibe o total de empresas processadas e a lista de empresas com erro
print(f"TOTAL DE EMPRESAS NO DICIONÁRIO DE PEG: {len(dicionario_de_peg.keys())}")
if empresas_com_erro:
    print(
        "Empresas que apresentaram erro:", set(empresas_com_erro)
    )  # Usa set para evitar duplicados
else:
    print("Nenhuma empresa apresentou erro.")

Processando empresa AALR3...
Erro ao calcular PEG para AALR3 na data 31/03/2016: '>' not supported between instances of 'str' and 'int'
TOTAL DE EMPRESAS NO DICIONÁRIO DE PEG: 0
Empresas que apresentaram erro: {'AALR3'}


In [17]:
dicionario_de_peg["AALR3"]

KeyError: 'AALR3'

In [ ]:
# Função para ler a planilha de balanço e extrair dados
def extrair_dados_balanco(arquivo_balanco):
    # Lê o arquivo de balanço em formato Excel
    balanco = pd.read_excel(
        arquivo_balanco, sheet_name=0, engine="openpyxl", header=None
    )
    # Identifica datas e indicadores
    datas = balanco.iloc[0, 1:].values  # Pega as datas da primeira linha (cabeçalhos)
    lucro_liquido = (
        balanco[balanco[0].str.contains("Lucro/Prejuízo do Período")].iloc[0, 1:].values
        * 1000
    )  # Convertendo para unidades
    patrimonio_liquido = (
        balanco[balanco[0].str.contains("Patrimônio Líquido")].iloc[0, 1:].values * 1000
    )  # Convertendo para unidades
    # Retorna os dados extraídos em forma de dicionário
    return {
        "datas": datas,
        "lucro_liquido": lucro_liquido,
        "patrimonio_liquido": patrimonio_liquido,
    }


# Função para obter o preço da ação e o número de ações via Yahoo Finance
def obter_dados_yahoo(empresa, datas):
    ticket = f"{empresa}.SA"
    acao = yf.Ticker(ticket)
    historico = acao.history(
        period="10y", auto_adjust=True
    )  # Pega 10 anos de histórico

    # Encontra o preço da ação mais próximo para cada data
    precos = []
    for data in datas:
        try:
            preco = historico.loc[data, "Close"]
        except KeyError:
            preco = historico[historico.index < data].iloc[-1][
                "Close"
            ]  # Pega o mais recente disponível
        precos.append(preco)

    numero_acoes = acao.info["sharesOutstanding"]
    return precos, numero_acoes


# Função para calcular PEG
def calcular_peg(preco, lpa_crescimento):
    return preco / lpa_crescimento if lpa_crescimento > 0 else None


# Função para calcular Graham
def calcular_graham(preco, lpa, vpa):
    valor_intrinseco = (22.5 * lpa * vpa) ** 0.5
    return valor_intrinseco


# Função principal para processar todas as empresas
def processar_empresas(lista_empresas, caminho_pasta_balancos):
    dicionario_dados = {}

    for empresa in lista_empresas:
        arquivo_balanco = os.path.join(caminho_pasta_balancos, f"{empresa}.xlsx")
        dados_balanco = extrair_dados_balanco(
            arquivo_balanco
        )  # Extrai dados do balanço
        datas = dados_balanco["datas"]

        precos, numero_acoes = obter_dados_yahoo(
            empresa, datas
        )  # Obtém preços da ação e número de ações

        # Calcula indicadores LPA, VPA, etc.
        lpa = [lucro / numero_acoes for lucro in dados_balanco["lucro_liquido"]]
        vpa = [
            patrimonio / numero_acoes
            for patrimonio in dados_balanco["patrimonio_liquido"]
        ]

        # Calcula crescimento do lucro
        crescimento_lpa = [
            (lpa[i] - lpa[i - 1]) / abs(lpa[i - 1]) if lpa[i - 1] != 0 else None
            for i in range(1, len(lpa))
        ]

        # Calcula PEG e Graham
        peg = [
            calcular_peg(preco, cresc)
            for preco, cresc in zip(precos[1:], crescimento_lpa)
        ]
        graham = [
            calcular_graham(preco, lpa_val, vpa_val)
            for preco, lpa_val, vpa_val in zip(precos, lpa, vpa)
        ]

        # Armazena os dados no dicionário
        dicionario_dados[empresa] = {
            "datas": datas,
            "lucro_liquido": dados_balanco["lucro_liquido"],
            "patrimonio_liquido": dados_balanco["patrimonio_liquido"],
            "precos": precos,
            "numero_acoes": numero_acoes,
            "lpa": lpa,
            "vpa": vpa,
            "crescimento_lpa": crescimento_lpa,
            "peg": peg,
            "graham": graham,
        }

    return dicionario_dados


# Exemplo de uso
BALANCOS_DEFINITIVO_FOLDER = "/caminho/para/balancos"
lista_empresas = ["PETR4", "VALE3"]  # Exemplo, deve vir do dicionário de balanços
dados_empresas = processar_empresas(lista_empresas, BALANCOS_DEFINITIVO_FOLDER)

# Visualizar resultados
for empresa, dados in dados_empresas.items():
    print(f"Empresa: {empresa}")
    print("Datas:", dados["datas"])
    print("PEG:", dados["peg"])
    print("Graham:", dados["graham"])